# Notebook - Workshop de Modelagem Preditiva

### Informações úteis

Para rodar os códigos no jupyter notebook, clicar na célula desejada e usar o atalho Shift+Enter (roda e vai para a próxima célula) ou Ctrl+Enter (roda).

O material necessário para esse notebook está todo disponível no repositório: https://analytics.fontes.intranet.bb.com.br/templates/workshop-modelagem-preditiva.git

Para copiar o conteúdo desse repositório no seu projeto na Plataforma Analítica:
1. Abra o terminal da Plataforma Analítica
2. Execute o comando abaixo no terminal da Plataforma

    git clone https://analytics.fontes.intranet.bb.com.br/templates/workshop-modelagem-preditiva.git

Dica: Os atalhos para copiar e colar no terminal da Plataformas são: Ctrl+Insert (copiar) e Shift+Insert (colar).

O notebook está organizado conforme as etapas do modelo CRISP-DM. 

### Configurar Ambiente e Importar Bibliotecas

Consutar bibliotecas e versões instaladas na Plataforma:

In [ ]:
!pip list

Neste notebook serão usadas algumas das bibliotecas mais comumente utilizadas em projetos de análise de dados. 

É preciso instalar as bibliotecas usadas que não estiverem listadas, o comando abaixo instala as bibliotecas seaborn e sklearn:

    !pip install seaborn sklearn --user

Após instalar bibliotecas recomenda-se reiniciar o kernel clicando no menu Kernel, opção Restart Kernel...

Com as bibliotecas necessárias já instaladas, é preciso carregá-las conforme código abaixo.

In [ ]:
import numpy as np # manipulação numérica
import pandas as pd # manipulação de dataframes (datasets)

import matplotlib.pyplot as plt # visualização de gráficos
import seaborn as sns # visualização de gráficos

from sklearn.impute import SimpleImputer # tratamento de valores ausentes
from sklearn.preprocessing import OneHotEncoder # tratamento de variáveis categóricas
from sklearn.compose import ColumnTransformer # definir tratamento para cada lista de variáveis

from sklearn.pipeline import Pipeline # definir sequência de cálculos do algoritmo de aprendizagem
from sklearn.model_selection import train_test_split # separar dados de treino e teste
from sklearn.model_selection import cross_val_score # validação cruzada
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score  # métricas de performance do modelo
from sklearn.model_selection import GridSearchCV # otimização de hiperparâmetros

# Algoritmos de aprendizagem
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import discriminant_analysis

from joblib import dump, load # salvar e carregar modelo

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

# Entendimento dos Dados

### Leitura e pré-visualização

In [ ]:
df = pd.read_csv('./UCI_Credit_Card_treino.csv', sep=',', decimal='.')
df.head()

In [ ]:
print('Quantidade de observações e colunas:', df.shape)
for v in df.columns:
    print(v)

In [ ]:
# Renomear algumas colunas
df = df.rename(columns={'default.payment.next.month': 'DEFAULT', 
                        'PAY_0': 'PAY_1'})

# Excluir coluna ID, pois não será utilizada para aprendizado do modelo
df.drop('ID', axis=1, inplace=True)

df.head()

### Classificação das variáveis por tipo

In [ ]:
# Variáveis Numéricas
vnum = ['LIMIT_BAL', 'AGE', 
        'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
        'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

# Variáveis Categóricas
vcat = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

# Variável target
vtgt = ['DEFAULT']

### Variável Target

In [ ]:
sns.countplot('DEFAULT', data=df, palette="Blues")
pct_default = round(df['DEFAULT'].sum()/len(df)*100)
plt.annotate(str(pct_default)+" %", xy=(0.7, 15000), xytext=(0.9, 8000), size=12)

Conclusões:
* A variável target é do tipo categórica e possui 2 categorias: 0 = cliente com pagamento em dia, 1 = cliente em descumprimento. Nesse caso, estamos diante de um problema de aprendizado supervisionado do tipo classificação.
* O percentual de clientes em descumprimento é menor que o de clientes com pagamento em dia. Quando alguma categoria é muito rara, menor que 5%, é recomendado realizar o balanceamento de classes. Nesse caso, não será necessário.

### Preditores numéricos

In [ ]:
df[vnum].describe()

Conclusões:
* LIMIT_BAL: clientes possuem limites entre 10 mil e 1 milhão.
* AGE: clientes possuem idade entre 21 e 79 anos.
* BILL_AMT: alguns clientes possuem faturas negativas, o que pode ser explicado por ressarcimentos, mas é ideal verificar. 
* PAY_AMT: o valor pago no mês anterior é sempre positivo.

In [ ]:
fig, axs = plt.subplots(7, 2, figsize = (20, 25), facecolor='white')  
for axs, v in zip(axs.flatten(), vnum):
    ax = sns.boxplot(x=v, y="DEFAULT", data=df, orient='h', palette="Blues", ax=axs)

Conclusões:
* LIMIT_BAL: clientes que ficam em default apresentam limites menores.
* AGE: a idade não parece ter relação com a target.
* BILL_AMT: clientes que ficam em default apresentam valores sensivelmente menores de fatura. 
* PAY_AMT: clientes que ficam em default apresentam pagamentos menores de fatura. 

### Preditores Categóricos

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (20,10), facecolor='white')
fig.suptitle('% DE CADA CATEGORIA')
for ax, v in zip(axs.flatten(), vcat):
    tab = pd.DataFrame(df[v].value_counts()/len(df))
    tab['x'] = tab.index
    ax = sns.barplot(data=tab, x='x', y=v, color='lightblue', ax=ax)    

Conclusões:
* SEX: a maioria dos clientes são mulheres (60%).
* EDUCATION: a maioria dos clientes possui graduação (50%), pós-graduação (35%) e ensino médio (15%). Obs: a categoria 0 não foi citada no dicionário de dados.
* MARRIAGE: a maioria dos clientes são solteiros (52%) seguido dos casados (45%). Obs: a categoria 0 não foi citada no dicionário.
* PAY: a maioria dos clientes está usando o crédito rotativo (50%), seguido por pagamento ok (20%) e sem consumo (10%), os demais são categorias em atraso. Obs: a categoria 1 = atrasado 1 mês só aparece no mês mais recente, o ideal seria verificar.

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (20,10), facecolor='white')
fig.suptitle('% DEFAULT EM CADA CATEGORIA')
for ax, v in zip(axs.flatten(), vcat):
    tab = df[['DEFAULT', v]].groupby([v]).mean()
    tab[v] = tab.index
    ax = sns.barplot(data=tab, x=v, y='DEFAULT', color='lightblue', ax=ax)    

Conclusões:
* SEX: as mulheres apresentaram taxa de default ligeiramente menor que a dos homens, 20% e 24% respectivamente.
* EDUCATION: os clientes cujo nível de instrução é outros ou desconhecido possuem menores taxas de default, seguido pelos clientes que possuem pós-graduação (18%).
* MARRIAGE: a taxa de default entre casados e solteiros é muito próxima, o estado civil não parece ter influência na taxa de fault.
* PAY: os clientes com faturas em atraso (categoria >= 2) apresentam as maiores taxas de default, o que já é esperado. Apesar de ser uma variável categórica, quanto mais próximo da categoria 8, maior a taxa de default, e quanto mais próximo da categoria -2, menor a taxa de default. Portanto, não será transformada em dummies e será utilizada como variável numérica. 

# Preparação dos Dados

### Classificação das Variáveis após Entendimento dos Dados

In [ ]:
# Variáveis Numéricas
vnum = ['LIMIT_BAL', 'AGE', 
        'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
        'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
        'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

# Variáveis Categóricas
vcat = ['SEX', 'EDUCATION', 'MARRIAGE']

# Variável Target
vtgt = ['DEFAULT']

### Tratamento de Valores Ausentes/Missing

O tratamento adequado depende de cada caso, alguns exemplos são:
* Variáveis com alto percentual de missing (acima de 10%) pode ser considerada a exclusão do aprendizado.
* Observações com alto percentual de missing (acima de 40%) pode ser considerada a exclusão do aprendizado. 
* Excluir observações com variável target missing nos dados de treino.
* Em variáveis numéricas, pode-se substituir os valores ausentes pela média/mediana dos valores válidos.
* Em variáveis categóricas, pode-se criar uma nova categoria para comportar os valores ausentes ou substituí-los pela categoria mais frequente.
* Em alguns casos pode ser útil aplicar um modelo preditivo para preencher os missing com valor estimado a partir dos outros preditores preenchidos.

In [ ]:
# Percentual de valores ausentes por variável
pct_miss = 100*df.isnull().sum()/len(df)
pct_miss[(-pct_miss).argsort()]

In [ ]:
# Método de imputação para variáveis numéricas -> substituir pela mediana
imp_num = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=True)

In [ ]:
# Visualizar variáveis numéricas antes da imputação
df[vnum].head()

In [ ]:
# Visualizar variáveis numéricas após imputação
pd.DataFrame(imp_num.fit_transform(df[vnum])).head()

### Tratamento de Preditores Categóricos

Novamente, o tratamento adequado depende do caso, alguns exemplos:
* Em variáveis com poucas categorias distintas, é comum criar variáveis indicadoras para cada categoria (dummies).
* Em variáveis com muitas categorias distintas, 20 ou mais, pode ser considerado outros métodos como criar agrupamentos de categorias que sejam semelhantes em relação à taxa de default, ou usar a própria taxa de default de cada categoria em substituição ao label da categoria (target_encoder):

        from category_encoders.target_encoder import TargetEncoder
        tge = TargetEncoder(handle_unknown='value', drop_invariant=True, min_samples_leaf=1)

In [ ]:
# Método para criar variáveis indicadoras para cada categoria
ohe = OneHotEncoder(handle_unknown='error', drop='first')

In [ ]:
# Visualizar variáveis antes do encoding
df[vcat].head()

In [ ]:
# Visualizar variáveis após encoding
pd.DataFrame(ohe.fit_transform(df[vcat]).toarray(), columns=ohe.get_feature_names(vcat)).head()

### Definir pré-processamento a ser aplicado a cada variável

Na célula abaixo será consolidado os diversos tratamentos aplicados a cada conjunto de variáveis. Para as variáveis numéricas a necessidade de imputação pela média nos valores ausentes e para as variáveis categóricas a criação de variáveis indicadoras.

In [ ]:
preprocessor = ColumnTransformer(transformers=[('vnum', imp_num, vnum),
                                               ('vcat', ohe, vcat)])

# Modelagem

### Separar dados de treino e teste

A base de treino será utilizada para treinamento e validação, enquando a base de teste será utilizada para avaliação do modelo, como uma forma de mensurar de forma independente do treinamento o qão bem o modelo irá performar nos novos dados.

In [ ]:
# Variáveis preditoras no objeto X
X = df[vnum + vcat]

# Variável Target no objeto y
y = df[vtgt]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

### Treinamento dos Algoritmos de Aprendizagem 

In [ ]:
algoritmos = [
    neighbors.KNeighborsClassifier(),
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.RandomForestClassifier(), 
    ensemble.GradientBoostingClassifier(),
    linear_model.LogisticRegression(),
    linear_model.RidgeClassifier()
]

In [ ]:
results = []
for alg in algoritmos:
    
    print('Rodando algoritmo:', alg.__class__.__name__)
    
    # Pipeline - sequencia de passos de pré-processamento e treinamento do algoritmo de aprendizagem
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', alg)])
    
    # Mensurar métrica de performance roc_auc usando validação cruzada
    measure_results = cross_val_score(my_pipeline, X_train, y_train, cv=5, scoring='roc_auc')
    results.append({'Estimator':alg.__class__.__name__, 'Score médio':measure_results.mean()})
    
    print('AUC da Validação Cruzada:', round(measure_results.mean(),2))

Conclusão:

O algoritmo de aprendizagem que apresentou melhor performance foi o GradientBoostingClassifier. A métrica escolhida para seleção do melhor modelo foi a área abaixo da curva ROC por ser uma medida adequada a problemas de classificação onde uma das categorias é mais rara que a outra, como é o caso do problema em análise.

### Otimização dos Hiperparâmetros

No etapa anterior, o treinamento dos algoritmos de aprendizagem foi realizado considerando os hiperparâmetros pré-definidos (default). Hiperparâmetros são parâmetros que controlam o treinamento do modelo. 

Cada algoritmo de aprendizagem possui seus hiperparâmetros específicos. Por exemplo, um dos hiperparâmetros do algoritmo GradientBoostingClassifier é a quantidade de classificadores (n_estimators) a ser adicionada. Portanto, o hiperparâmetro n_estimators controla como o modelo será treinado. Quanto menor o valor de n_estimators, maior a tendência de underfitting, quanto maior o valor de n_estimators, maior a tendência de overfitting. Então, como definir o valor ideal de n_estimators?

In [ ]:
# Necessário preprocessar dados antes
X_train_ = preprocessor.fit_transform(X_train)

# Algoritmo a ser otimizado: KNN
alg = ensemble.GradientBoostingClassifier()

# Definir valores de K a serem testados
n_estimators = list(range(5,55,5))
hiperparametros = dict(n_estimators=n_estimators)

# Treinar modelo para os valores de K
gs = GridSearchCV(alg, hiperparametros, cv=5, scoring='accuracy', return_train_score=True).fit(X_train_, y_train)

# Gráfico com resultados
plt.plot(n_estimators, gs.cv_results_['mean_test_score'], label="AUC Teste")
plt.plot(n_estimators, gs.cv_results_['mean_train_score'], label="AUC Treino")
leg = plt.legend(loc='lower right', ncol=1, shadow=True, fancybox=True)

Conclusão:

A partir de 10 estimadores a performance no teste da validação cruzada não tem mais ganho, apesar da performance no treino melhorar adicionando mais estimadores. Portanto, o valor ideal seria n_estimators=10.

A seguir, vamos treinar o modelo com os valores ideais dos hiperparâmetros e salvar o resultado contento as intruções para aplicação do modelo a novos dados em um arquivo .joblib.

### Salvar Modelo Selecionado

In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                              ('model', ensemble.GradientBoostingClassifier(n_estimators=10))])
my_pipeline.fit(X_train, y_train) 
dump(my_pipeline, './modelo_selecionado.joblib')

# Avaliação

Uma estimativa de quão bem o modelo preditivo irá performar diante de novos dados, independentes daqueles utilizados para treinamento, é calculada atráves da aplicação do modelo aos dados de teste.

Diante de um problema de classificação as previsões se referem à probabiblidade da observação pertencer à classe positiva, que no caso é o cliente em default. Podemos escolher diversos pontos de corte entre 0 e 1 para decidir quais clientes serão previstos como default (prob > ponto de corte) e quais serão previstos como ok (prob < ponto de corte).

Dado que há uma expectativa da área de negócio em identificar ao menos 70% dos clientes que entrarão em default, vamos ajustar o ponto de corte para que esse percentual seja atingido. Porém precisamos avaliar se esse ponto de corte não impactará em muitos FP, clientes previstos como default e que não se tornaram inadimplentes.

In [ ]:
pct_ponto_corte = 0.40
probs = my_pipeline.predict_proba(X_test)[:,1]
ponto_corte = np.quantile(probs, 1-pct_ponto_corte)
VN, FP, FN, VP = confusion_matrix(y_test, (probs>ponto_corte).astype('int')).ravel()*100/len(y_test)
print('Matriz de Confusão: \n', [VP, FN], '\n', [FP, VN])
print('')
print('Acurácia:', round(accuracy_score(y_test, (probs>ponto_corte).astype('int')),2))
print('Recall:', round(recall_score(y_test, (probs>ponto_corte).astype('int')),2))
print('Precisão:', round(precision_score(y_test, (probs>ponto_corte).astype('int')),2))
print('')
print('Ponde de Corte:', round(ponto_corte,2))

Conclusão:

Para atingir a métrica de sucesso de identificar 70% dos clientes inadimplentes o modelo indica 40% do clientes como potenciais inadimplentes. Para isso, a taxa de falsos positivos, cliente previstos como inadimplentes que honraram ses compromissos no mÊs seguinte, fica em torno de 25%, o que pode ser considerado um valor alto pela área de negócio e representar perdas de oportunidades devido a ter mais cautela em negócios com esses clientes.

Portanto, pode ser necessário mais uma iteração de levantamento de dados e refinamento do modelo para alcançar uma performance melhor do que obtida até o momento.

Caso esse modelo seja aprovado, Ao aplicar o modelo e obter as previsões, o ponto de corte ideal dos scores para identificar clientes que se tornaram inadimplentes será considerar aqueles com score maior que 0.19. 

# Aplicação

### Leitura e pré-visualização dos novos dados

In [ ]:
dfnew = pd.read_csv('./UCI_Credit_Card_novo.csv', sep=',', decimal='.')
dfnew.head()

Note que para os novos dados não temos a coluna da variável target, pois não temos essa informação ainda. Por isso, vamos aplicar o modelo treinado para fazer essa previsão.

Observação: as alterações feitas nos dados antes do pré-processamento precisam ser replicadas aos novos dados. Neste caso, a única alteração foi renomear a coluna PAY_0 para PAY_1.

In [ ]:
dfnew = dfnew.rename(columns={'PAY_0': 'PAY_1'})

### Carga do arquivo que contém o modelo preditivo e aplicação aos novos dados

In [ ]:
my_pipeline = load('./modelo_selecionado.joblib')

In [ ]:
# Dataset para receber dados escorados
dfscr = dfnew[['ID']]

# Coluna com as probabilidades calculados
dfscr['probs'] = my_pipeline.predict_proba(dfnew)[:,1]

# Coluna com as previsões a partir do ponto de corte definido
dfscr['previsto'] = [1 if x>0.19 else 0 for x in dfscr['probs']]

# Pré-visualizar dados escorados
dfscr.head()

# Acabou?!

Ainda não! Depois de colocar seu modelo em produção para gerar previsões e fornecer insumos para as tomadas de decisão, é preciso acompanhar e monitorar a performance do modelo! 

No caso em estudo, no mês seguinte será possível observar quais clientes realmente entraram em default. É preciso deixar uma rotina de monitoramento que compare os valores previstos pelo modelo com os valores observados após o período de performance da variável target. Dessa forma, será possível acompanhar a performance do modelo ao longo do tempo e identificar quando a qualidade das previsões começa a ficar degradada. 

Nesse momento, é hora de pensar em revisitar o modelo novamente. A solução pode ser recalibrar o modelo com dados mais recentes, ou fazer um novo modelo, caso haja necessidade de levantar novas variáveis ou alterar a estrutura do modelo. Também há possibilidade de colocar em produção modelos de machine learning que já preveem a recalibragem automática ao longo do tempo.

# Agora sim, acabou !!!